In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def add_layer(inputs, in_size, out_size, nth_layer, activation_function=None):
    layer_name = "layer%s" % nth_layer
    with tf.name_scope("layer"):
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random_uniform([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)
        
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
            
        with tf.name_scope('Wx_pluse_b'):
            Wx_pluse_b = tf.add(tf.matmul(inputs, Weights), biases)
        
        if activation_function == None:
            outputs = Wx_pluse_b
        else:
            outputs = activation_function(Wx_pluse_b)
        
        tf.summary.histogram(layer_name + 'outputs', outputs)
        
        return outputs

In [3]:
x_data = np.linspace(-1,1,300)[:,np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise
y_data

array([[ 0.50856667],
       [ 0.50959252],
       [ 0.4980469 ],
       [ 0.49994706],
       [ 0.47170675],
       [ 0.40781388],
       [ 0.48110933],
       [ 0.38479345],
       [ 0.41606389],
       [ 0.37749565],
       [ 0.37986524],
       [ 0.35959481],
       [ 0.42867627],
       [ 0.41171553],
       [ 0.18682417],
       [ 0.28858489],
       [ 0.16644271],
       [ 0.2745607 ],
       [ 0.23827867],
       [ 0.3081676 ],
       [ 0.26709169],
       [ 0.20718224],
       [ 0.23075176],
       [ 0.22261726],
       [ 0.13130188],
       [ 0.17744671],
       [ 0.12890994],
       [ 0.23988622],
       [ 0.01910367],
       [ 0.17093307],
       [ 0.10244213],
       [ 0.13586436],
       [ 0.03267581],
       [ 0.11483321],
       [ 0.11745671],
       [ 0.01134738],
       [ 0.11658773],
       [ 0.0337604 ],
       [ 0.06399781],
       [-0.02504961],
       [ 0.05453265],
       [-0.01391911],
       [ 0.01047356],
       [-0.00623312],
       [ 0.01974636],
       [-0

In [4]:
# define placeholder
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

In [5]:
layer1 = add_layer(xs, 1, 10, nth_layer=1, activation_function=tf.nn.relu)
prediction = add_layer(layer1, 10, 1, nth_layer=2, activation_function=None)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("trains"):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

In [6]:
sess = tf.Session()
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("logs/", sess.graph)
sess.run(tf.global_variables_initializer())

In [7]:
for i in range(1000):
    sess.run(train_step, feed_dict={xs:x_data, ys:y_data})
    if i % 100 == 0:
        result = sess.run(merged, feed_dict={xs:x_data, ys:y_data})
        writer.add_summary(result, i)